In [1]:
.libPaths(R.home('/opt/homebrew/Caskroom/miniforge/base/lib/R/library'))

In [2]:
Output = '/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1015. Fire Sufficient Similarity/P1015.3. Analyses/P1015.3.2. Data Processing/Output'
cur_date = '070825'

library(missForest)
library(readxl)
library(openxlsx)
library(tidyverse)
library(factoextra)

# reading in files
ws_df = data.frame(Sample = 'WS', read_excel("Input/Woodsmoke_Data_053125.xlsx", sheet = 2)) %>%
    select(-Number)
wf_df = data.frame(Sample = 'WF', read_excel("Input/Wildfire_Data_053125.xlsx", sheet = 2)) %>%
    select(-Number)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Welcome! Want to learn more? See two factoextra-related books at https://goo.gl/ve3WBa



In [3]:
head(ws_df)
head(wf_df)

,Sample,HAWC_ID,Study,Sample_Number,Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WS,429174,Arif et al. 2017,1,PAH,Weight,DTXSID6025648,1-Methylphenanthrene,NA
2,WS,429174,Arif et al. 2017,1,PAH,Weight,DTXSID0025654,1-Methylpyrene,NA
3,WS,429174,Arif et al. 2017,1,PAH,Weight,DTXSID4020878,2-Methylnaphthalene,NA
4,WS,429174,Arif et al. 2017,1,PAH,Weight,DTXSID4063030,2-Methylpyrene,NA
5,WS,429174,Arif et al. 2017,1,PAH,Weight,DTXSID50187178,4-Methylpyrene,NA
6,WS,429174,Arif et al. 2017,1,PAH,Weight,DTXSID6063143,5-Methylchrysene,NA


,Sample,HAWC_ID,Study,Sample_Number,Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WF,1289737,Campbell et al. 2024,1,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA
2,WF,1289737,Campbell et al. 2024,1,PAH,Weight,DTXSID9020877,1-Methylnaphthalene,NA
3,WF,1289737,Campbell et al. 2024,1,PAH,Weight,DTXSID0029187,"2,6-Dimethylnaphthalene",NA
4,WF,1289737,Campbell et al. 2024,1,PAH,Weight,DTXSID4020878,2-Methylnaphthalene,NA
5,WF,1289737,Campbell et al. 2024,1,PAH,Weight,DTXSID3061134,9-Methylanthracene,NA
6,WF,1289737,Campbell et al. 2024,1,PAH,Weight,DTXSID3023845,Acenaphthylene,NA


In [4]:
dim(ws_df)
dim(wf_df)

[1] 4391    9

[1] 3795    9

Starting with 4391 WS records and 3795 WF records. 

In [5]:
# combining dfs
combined_df = rbind(ws_df, wf_df)
           
head(combined_df)

,Sample,HAWC_ID,Study,Sample_Number,Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WS,429174,Arif et al. 2017,1,PAH,Weight,DTXSID6025648,1-Methylphenanthrene,NA
2,WS,429174,Arif et al. 2017,1,PAH,Weight,DTXSID0025654,1-Methylpyrene,NA
3,WS,429174,Arif et al. 2017,1,PAH,Weight,DTXSID4020878,2-Methylnaphthalene,NA
4,WS,429174,Arif et al. 2017,1,PAH,Weight,DTXSID4063030,2-Methylpyrene,NA
5,WS,429174,Arif et al. 2017,1,PAH,Weight,DTXSID50187178,4-Methylpyrene,NA
6,WS,429174,Arif et al. 2017,1,PAH,Weight,DTXSID6063143,5-Methylchrysene,NA


In [6]:
length(unique(combined_df$Name))
length(unique(ws_df$Name))
length(unique(wf_df$Name))

[1] 151

[1] 93

[1] 114

114 unique metals/chemicals total.

# Filter #1: Chemical/Metal Background Filters

Determining what variables have missing data for all variables delineated based on the dfs the data will be split into and analyzed. 

A variable (ie. `Name` which represents a chemical or metal) will be retained if >= 20% of its experimental data points were measurable values. In other words, chemicals or metals were excluded if they had NAs (not evaluated) or had non-detect values across >= 80% of the samples.

In [7]:
`%notin%` <- Negate(`%in%`)

sample_type_presence_df = combined_df %>%
    # if an experimental value is there count it as being "present"
    mutate(count = ifelse(Value %notin% c('NA', 'ND'), 1, 0)) %>%
    # determining which have at least 20% experimental values for each chemical/metal
    group_by(Metric, Sample, Name) %>%
    # calculating the percentage of experimental data points
    reframe(Variable_Presence_Percentage = (sum(count)/n()) * 100) %>%
    filter(Variable_Presence_Percentage >= 40) 

head(sample_type_presence_df)

Metric,Sample,Name,Variable_Presence_Percentage
<chr>,<chr>,<chr>,<dbl>
Volume,WF,"1,2,4-Trimethylbenzene",100.00000
Volume,WF,Acenaphthylene,100.00000
Volume,WF,Anthracene,42.10526
Volume,WF,Arsenic,100.00000
Volume,WF,Benz(a)anthracene,100.00000
Volume,WF,Benzene,100.00000


In [8]:
# filtering the original df
filter1_df = inner_join(sample_type_presence_df[,1:3], combined_df) 

head(filter1_df)

Joining with `by = join_by(Metric, Sample, Name)`


Metric,Sample,Name,HAWC_ID,Study,Sample_Number,Class,DTXSID,Value
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>
Volume,WF,"1,2,4-Trimethylbenzene",1289926,Wang et al. 2024,1,VOC,DTXSID6021402,730
Volume,WF,"1,2,4-Trimethylbenzene",1306371,Ketcherside et al. 2024,1,VOC,DTXSID6021402,130
Volume,WF,Acenaphthylene,1289895,Silberstein et al. 2023,1,PAH,DTXSID3023845,7.0000000000000007E-2
Volume,WF,Acenaphthylene,1285830,Navarro et al. 2019,1,PAH,DTXSID3023845,1
Volume,WF,Acenaphthylene,1285830,Navarro et al. 2019,2,PAH,DTXSID3023845,1
Volume,WF,Acenaphthylene,1285830,Navarro et al. 2019,3,PAH,DTXSID3023845,1


In [9]:
dim(combined_df)
dim(filter1_df)

[1] 8186    9

[1] 5352    9

Started with 8186 records, 1638 were removed, leaving 6548. 

# Filter #2: Study Background Filter

A study will be retained if >= 20% of its experimental data points were measurable values. In other words, chemicals were excluded if they were NAs (not evaluated) or had non-detect values across < 20% of the samples.

In [10]:
study_type_presence_df = filter1_df %>%
    # if an experimental value is there count it as being "present"
    mutate(count = ifelse(Value %notin% c('NA', 'ND'), 1, 0)) %>%
    # determining which have at least 20% experimental values for each chemical/metal
    group_by(Metric, Class, HAWC_ID) %>%
    # calculating the percentage of experimental data points
    reframe(Study_Presence_Percentage = (sum(count)/n()) * 100) %>%
    filter(Study_Presence_Percentage >= 40) 

head(study_type_presence_df)

Metric,Class,HAWC_ID,Study_Presence_Percentage
<chr>,<chr>,<dbl>,<dbl>
Volume,Metal,266980,100
Volume,Metal,266987,100
Volume,Metal,452289,100
Volume,Metal,885041,100
Volume,Metal,993956,100
Volume,Metal,1070627,100


In [11]:
filter2_df = inner_join(study_type_presence_df[,1:3], filter1_df)

head(filter2_df)

Joining with `by = join_by(Metric, Class, HAWC_ID)`


Metric,Class,HAWC_ID,Sample,Name,Study,Sample_Number,DTXSID,Value
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
Volume,Metal,266980,WS,Lead,Stockfelt et al. 2012,1,Pb,170
Volume,Metal,266980,WS,Lead,Stockfelt et al. 2012,2,Pb,400
Volume,Metal,266980,WS,Potassium,Stockfelt et al. 2012,1,K,9700
Volume,Metal,266980,WS,Potassium,Stockfelt et al. 2012,2,K,8800
Volume,Metal,266980,WS,Zinc,Stockfelt et al. 2012,1,Zn,2400
Volume,Metal,266980,WS,Zinc,Stockfelt et al. 2012,2,Zn,3100


In [12]:
dim(filter1_df)
dim(filter2_df)

[1] 5352    9

[1] 4895    9

122 more records were removed.

# Filter #3: Chemical/Metal Background Filter

In the intial filter, WS and WF samples were combined. However, this time variables (ie. metal or chemical) will be split based on their `Metric` (ie. volume or weight) and then retained if that variable is found in both woodsmoke and wildfire samples within volume or weight samples.

In [13]:
split_filtered_df = filter2_df %>%
    group_by(Sample, Metric) %>%
    group_split

split_ws_vol_df = split_filtered_df[[3]]
split_ws_weight_df = split_filtered_df[[4]]
split_wf_vol_df = split_filtered_df[[1]]
split_wf_weight_df = split_filtered_df[[2]]

In [14]:
# seeing how many unique variables are in each df and if they're consistent in each file
length(unique(split_ws_vol_df$Name))
length(unique(split_wf_vol_df$Name))
length(unique(split_ws_weight_df$Name))
length(unique(split_wf_weight_df$Name))

[1] 77

[1] 55

[1] 41

[1] 79

In [15]:
# they're not so first getting variables that are in weight or volume samples
consistent_wf_vol_df = split_wf_vol_df %>%
    filter(Name %in% unique(split_ws_vol_df$Name))
consistent_ws_vol_df = split_ws_vol_df %>%
    filter(Name %in% consistent_wf_vol_df$Name)
consistent_wf_weight_df = split_wf_weight_df %>%
    filter(Name %in% unique(split_ws_weight_df$Name))
consistent_ws_weight_df = split_ws_weight_df %>%
    filter(Name %in% consistent_wf_weight_df$Name)

length(unique(consistent_wf_vol_df$Name))
length(unique(consistent_ws_vol_df$Name))
length(unique(consistent_wf_weight_df$Name))
length(unique(consistent_ws_weight_df$Name))

[1] 35

[1] 35

[1] 29

[1] 29

There were 77, 57, 51 and 89 woodsmoke weight, woodsmoke volume, wildfire weight, and wildfire volume samples, respectively. 37 variables were common between volume samples and 37 were common between weight records and will be retained.

In [16]:
# recombining data
vol_df = rbind(consistent_ws_vol_df, consistent_wf_vol_df)
weight_df = rbind(consistent_ws_weight_df, consistent_wf_weight_df)

head(vol_df)

Metric,Class,HAWC_ID,Sample,Name,Study,Sample_Number,DTXSID,Value
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
Volume,Metal,266980,WS,Lead,Stockfelt et al. 2012,1,Pb,170
Volume,Metal,266980,WS,Lead,Stockfelt et al. 2012,2,Pb,400
Volume,Metal,266980,WS,Potassium,Stockfelt et al. 2012,1,K,9700
Volume,Metal,266980,WS,Potassium,Stockfelt et al. 2012,2,K,8800
Volume,Metal,266980,WS,Zinc,Stockfelt et al. 2012,1,Zn,2400
Volume,Metal,266980,WS,Zinc,Stockfelt et al. 2012,2,Zn,3100


# Square Root Imputation

Taking the square root of the min value to impute MNAR data.

In [17]:
# first determining number of ND and NA values in each df
vol_df %>%
    filter(Value %in% c('NA', 'ND')| grepl("<", Value)) %>%
    group_by(Value) %>%
    summarize(n())

weight_df %>%
    filter(Value %in% c('NA', 'ND')| grepl("<", Value)) %>%
    group_by(Value) %>%
    summarize(n())

Value,n()
<chr>,<int>
NA,36
ND,25


Value,n()
<chr>,<int>
<0.00025,1
<0.0005,6
<0.0006,2
<0.0007,3
<0.04,1
NA,276
ND,127


In [18]:
dim(vol_df)
dim(weight_df)

[1] 1013    9

[1] 2270    9

In [19]:
# creating a df for MAR data
mar_weight_df = weight_df %>%
    filter(Value == 'NA')
mar_vol_df = vol_df %>%
    filter(Value == 'NA')

# creating dfs for studies that contain MNAR data
preimputed_lod_weight_df = anti_join(weight_df, mar_weight_df) 
preimputed_vol_df = anti_join(vol_df, mar_vol_df) 

Joining with `by = join_by(Metric, Class, HAWC_ID, Sample, Name, Study,
Sample_Number, DTXSID, Value)`
Joining with `by = join_by(Metric, Class, HAWC_ID, Sample, Name, Study,
Sample_Number, DTXSID, Value)`


In [20]:
# first imputing non detect values for studies that had a limit of detection (LOD) provided
lod_values_df = preimputed_lod_weight_df %>%
    filter(grepl("<", Value)) %>%
    separate(Value, sep = '<', into = c(NA, 'Value')) %>%
    mutate(Value = sqrt(as.numeric(Value)))

dim(lod_values_df)
head(lod_values_df)

# adding these newly imputed values back into the df
preimputed_weight_df = rbind(anti_join(preimputed_lod_weight_df, lod_values_df[,1:8]), lod_values_df)

[1] 13  9

Metric,Class,HAWC_ID,Sample,Name,Study,Sample_Number,DTXSID,Value
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>
Weight,PAH,821855,WS,Naphthalene,Erlandsson et al. 2020,1,DTXSID8020913,0.20000000
Weight,PAH,1045457,WS,"Dibenzo(a,h)anthracene",Popadic 2018,1,DTXSID9020409,0.01581139
Weight,PAH,1289739,WF,Benzo(GHI)perylene,Campos et al. 2019,2,DTXSID5023908,0.02645751
Weight,PAH,1289739,WF,Benzo(GHI)perylene,Campos et al. 2019,6,DTXSID5023908,0.02645751
Weight,PAH,1289739,WF,Benzo(GHI)perylene,Campos et al. 2019,8,DTXSID5023908,0.02645751
Weight,PAH,1289739,WF,Benzo(a)pyrene,Campos et al. 2019,8,DTXSID2020139,0.02449490


Joining with `by = join_by(Metric, Class, HAWC_ID, Sample, Name, Study,
Sample_Number, DTXSID)`


In [21]:
preimputed_weight_df$Value = as.numeric(preimputed_weight_df$Value)
preimputed_vol_df$Value = as.numeric(preimputed_vol_df$Value)
mar_weight_df$Value = as.numeric(mar_weight_df$Value)
mar_vol_df$Value = as.numeric(mar_vol_df$Value)

# reordering cols
preimputed_weight_df = preimputed_weight_df[,c(3,1,4,6,7,2,8,5,9)] %>%
    # normalizing data
    mutate(Value = log2(Value + 1))
preimputed_vol_df = preimputed_vol_df[,c(3,1,4,6,7,2,8,5,9)] %>%
    # normalizing data
    mutate(Value = log2(Value + 1))

head(preimputed_weight_df)

Warning message:
“NAs introduced by coercion”
Warning message:
“NAs introduced by coercion”
Warning message:
“NAs introduced by coercion”
Warning message:
“NAs introduced by coercion”


HAWC_ID,Metric,Sample,Study,Sample_Number,Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
267105,Weight,WS,Danielsen et al. 2011,1,Metal,Cd,Cadmium,4.925999
267105,Weight,WS,Danielsen et al. 2011,2,Metal,Cd,Cadmium,NA
267105,Weight,WS,Danielsen et al. 2011,1,Metal,Cr,Chromium,4.485427
267105,Weight,WS,Danielsen et al. 2011,2,Metal,Cr,Chromium,4.478972
267105,Weight,WS,Danielsen et al. 2011,1,Metal,Cu,Copper,6.057450
267105,Weight,WS,Danielsen et al. 2011,2,Metal,Cu,Copper,5.145677


In [22]:
# creating a df for MNAR data
mnar_weight_df = preimputed_weight_df %>%
    filter(is.na(Value))
mnar_vol_df = preimputed_vol_df %>%
    filter(is.na(Value))

head(mnar_weight_df)

HAWC_ID,Metric,Sample,Study,Sample_Number,Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
267105,Weight,WS,Danielsen et al. 2011,2,Metal,Cd,Cadmium,NA
267105,Weight,WS,Danielsen et al. 2011,1,Metal,Mn,Manganese,NA
267105,Weight,WS,Danielsen et al. 2011,2,Metal,Mn,Manganese,NA
271638,Weight,WS,Jalava et al. 2012,1,Metal,Pb,Lead,NA
271638,Weight,WS,Jalava et al. 2012,2,Metal,Pb,Lead,NA
271638,Weight,WS,Jalava et al. 2012,3,Metal,Pb,Lead,NA


In [23]:
square_root_imputation = function(mnar_df, preimputed_df){
    # """
    # Imputing MNAR values based on the square root of the minimum concentration for each chemical 
    # within a study or metal.
    
    # :param: MNAR dataframe, preimputed dataframe
    # :output: a dataframe with imputed values

    # """

    id_df = mnar_df[is.na(mnar_df$Value),]
    study_ids = unique(id_df$HAWC_ID)
    
    imputed_df = data.frame()
    for (i in 1:length(id_df$HAWC_ID)){
    
        filtered_df = preimputed_df %>%
            filter(HAWC_ID == id_df$HAWC_ID[i], DTXSID == id_df$DTXSID[i])
    
        na_df = filtered_df %>%
            filter(is.na(Value)) %>%
            mutate(Value = sqrt(min(filtered_df$Value, na.rm = TRUE)))
    
        # combining rows of data
        combined_df = rbind(filtered_df %>%
            filter(!is.na(Value)), na_df)
    
        imputed_df = rbind(imputed_df, combined_df) %>%
                        unique()
    }
    return(imputed_df)    
    
}

# calling fn
imputed_vol_df = square_root_imputation(mnar_vol_df, preimputed_vol_df) %>%
    mutate(Value = ifelse(Value == Inf, NA, Value))
imputed_weight_df = square_root_imputation(mnar_weight_df, preimputed_weight_df) %>%
    mutate(Value = ifelse(Value == Inf, NA, Value))
    
head(imputed_weight_df)

Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `Value = sqrt(min(filtered_df$Value, na.rm = TRUE))`.
Caused by warning in `min()`:
! no non-missing arguments to min; returning Inf”
Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `Value = sqrt(min(filtered_df$Value, na.rm = TRUE))`.
Caused by warning in `min()`:
! no non-missing arguments to min; returning Inf”
Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `Value = sqrt(min(filtered_df$Value, na.rm = TRUE))`.
Caused by warning in `min()`:
! no non-missing arguments to min; returning Inf”
Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `Value = sqrt(min(filtered_df$Value, na.rm = TRUE))`.
Caused by warning in `min()`:
! no non-missing arguments to min; returning Inf”
Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `Value = sqrt(min(filtered_df$Value, na.rm = TRUE))`.
Caused by warning in `min()`:
! no non-missing arguments to min; retur

HAWC_ID,Metric,Sample,Study,Sample_Number,Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
267105,Weight,WS,Danielsen et al. 2011,1,Metal,Cd,Cadmium,4.925999
267105,Weight,WS,Danielsen et al. 2011,2,Metal,Cd,Cadmium,2.219459
267105,Weight,WS,Danielsen et al. 2011,1,Metal,Mn,Manganese,NA
267105,Weight,WS,Danielsen et al. 2011,2,Metal,Mn,Manganese,NA
271638,Weight,WS,Jalava et al. 2012,1,Metal,Pb,Lead,NA
271638,Weight,WS,Jalava et al. 2012,2,Metal,Pb,Lead,NA


In [24]:
# there are still a few variables that didn't have any values to do the square root imputation
# their imputed values will be based off of distributions from other studies
other_studies_vol_df = semi_join(preimputed_vol_df %>%
                filter(!is.na(Value)), imputed_vol_df %>%
                               filter(is.na(Value)) %>%
    select(2,3,6,7)) %>%
    group_by(DTXSID) %>%
    summarize(Value = sqrt(min(Value)))

other_studies_weight_df = semi_join(preimputed_weight_df %>%
                filter(!is.na(Value)), imputed_weight_df %>%
                               filter(is.na(Value)) %>%
    select(2,3,6,7)) %>%
    group_by(DTXSID) %>%
    summarize(Value = sqrt(min(Value)))

head(other_studies_vol_df)
dim(other_studies_vol_df)
head(other_studies_weight_df)
dim(other_studies_weight_df)

Joining with `by = join_by(Metric, Sample, Class, DTXSID)`
Joining with `by = join_by(Metric, Sample, Class, DTXSID)`


DTXSID,Value
<chr>,<dbl>
DTXSID0022432,0.28993838
DTXSID0023878,0.07867839
DTXSID2020139,0.10920141
DTXSID3024104,0.16902412
DTXSID6024254,0.11981358


[1] 5 2

DTXSID,Value
<chr>,<dbl>
Cr,0.37081468
DTXSID0023907,0.10382600
DTXSID3023845,0.13868023
DTXSID8020913,0.17944249
DTXSID8024153,0.07455255
Fe,0.78328261


[1] 9 2

7 (volume) and 14 (weight) samples were imputed based on values from other studies.

In [25]:
# removing NAs from the imputed df 
removed_imputed_vol_df = imputed_vol_df %>%
    filter(!is.na(Value))
removed_imputed_weight_df = imputed_weight_df %>%
    filter(!is.na(Value))

# adding in the newly imputed values back in 
all_imputed_vol_df = left_join(imputed_vol_df %>%
        filter(is.na(Value)) %>%
        select(-Value), other_studies_vol_df)

all_imputed_weight_df = left_join(imputed_weight_df %>%
        filter(is.na(Value)) %>%
        select(-Value), other_studies_weight_df)

all_imputed_vol_df = rbind(removed_imputed_vol_df, all_imputed_vol_df)
all_imputed_weight_df = rbind(removed_imputed_weight_df, all_imputed_weight_df)

head(all_imputed_weight_df)

Joining with `by = join_by(DTXSID)`
Joining with `by = join_by(DTXSID)`


HAWC_ID,Metric,Sample,Study,Sample_Number,Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
267105,Weight,WS,Danielsen et al. 2011,1,Metal,Cd,Cadmium,4.925999
267105,Weight,WS,Danielsen et al. 2011,2,Metal,Cd,Cadmium,2.219459
1040882,Weight,WS,Nordin et al. 2015,1,Metal,Ca,Calcium,12.797864
1040882,Weight,WS,Nordin et al. 2015,2,Metal,Ca,Calcium,3.577410
1040882,Weight,WS,Nordin et al. 2015,1,Metal,Fe,Iron,10.195987
1040882,Weight,WS,Nordin et al. 2015,2,Metal,Fe,Iron,3.193116


In [26]:
# creating 1 df
mnar_imputed_vol_df = rbind(#imputed data
                               all_imputed_vol_df,
                               # adding in MAR data
                               mar_vol_df,
                               # adding back in non-imputed data
                               anti_join(preimputed_vol_df, all_imputed_vol_df[,1:8]))
mnar_imputed_weight_df = rbind(# imputed data
                               all_imputed_weight_df,
                               # adding in MAR data
                               mar_weight_df,
                               # adding back in non-imputed data
                               anti_join(preimputed_weight_df, all_imputed_weight_df[,1:8]))

head(mnar_imputed_weight_df)
dim(mnar_imputed_vol_df)
dim(mnar_imputed_weight_df)

Joining with `by = join_by(HAWC_ID, Metric, Sample, Study, Sample_Number,
Class, DTXSID, Name)`
Joining with `by = join_by(HAWC_ID, Metric, Sample, Study, Sample_Number,
Class, DTXSID, Name)`


HAWC_ID,Metric,Sample,Study,Sample_Number,Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
267105,Weight,WS,Danielsen et al. 2011,1,Metal,Cd,Cadmium,4.925999
267105,Weight,WS,Danielsen et al. 2011,2,Metal,Cd,Cadmium,2.219459
1040882,Weight,WS,Nordin et al. 2015,1,Metal,Ca,Calcium,12.797864
1040882,Weight,WS,Nordin et al. 2015,2,Metal,Ca,Calcium,3.577410
1040882,Weight,WS,Nordin et al. 2015,1,Metal,Fe,Iron,10.195987
1040882,Weight,WS,Nordin et al. 2015,2,Metal,Fe,Iron,3.193116


[1] 1013    9

[1] 2270    9

# Filter #4: Double Checking MAR Records

Making sure that when the data is split based on `Class` and transformed into a wide format, we have at least 20% of the records with numeric values for each metal or chemical. Converting values to 0s and 1s to count number of experimental and NDs previously imputed. Chemicals/metals that don't have at least 20% of values present will be removed.

In [27]:
wider_vol_df = mnar_imputed_vol_df %>%
    # creating sample ids
    unite("SampleID", c("Sample", "HAWC_ID", "Sample_Number"), remove = FALSE) %>%
    select(-c("Name", "Class")) %>%
    pivot_wider(names_from = "DTXSID", values_from = "Value") 

wider_weight_df = mnar_imputed_weight_df %>%
    # creating sample ids
    unite("SampleID", c("Sample", "HAWC_ID", "Sample_Number"), remove = FALSE) %>%
    select(-c("Name", "Class")) %>%
    pivot_wider(names_from = "DTXSID", values_from = "Value")

head(wider_weight_df)

SampleID,HAWC_ID,Metric,Sample,Study,Sample_Number,Cd,Ca,Fe,Cr,⋯,DTXSID3024289,Mn,Ni,Cu,Zn,K,Mg,DTXSID1025649,DTXSID4075459,DTXSID8074819
<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
WS_267105_1,267105,Weight,WS,Danielsen et al. 2011,1,4.925999,NA,10.370687,4.485427,⋯,8.618386,0.5244111,3.9726927,6.057450,10.009829,NA,NA,3.459432,NA,NA
WS_267105_2,267105,Weight,WS,Danielsen et al. 2011,2,2.219459,NA,10.792790,4.478972,⋯,7.942515,0.5244111,3.4982509,5.145677,9.552669,NA,NA,5.247928,NA,NA
WS_1040882_1,1040882,Weight,WS,Nordin et al. 2015,1,5.426265,12.79786,10.195987,7.312883,⋯,NA,0.5244111,0.2065051,8.396605,12.829921,15.12771,10.81938,NA,NA,NA
WS_1040882_2,1040882,Weight,WS,Nordin et al. 2015,2,5.426265,3.57741,3.193116,6.658211,⋯,NA,0.5244111,0.2065051,7.622052,10.995060,13.16757,10.79523,NA,NA,NA
WS_1063054_1,1063054,Weight,WS,Corsini et al. 2013,1,NA,11.32249,7.787903,5.357552,⋯,NA,8.1344263,4.3923174,8.134426,12.372049,NA,NA,NA,NA,NA
WS_1063054_2,1063054,Weight,WS,Corsini et al. 2013,2,NA,12.60061,8.939579,2.314639,⋯,NA,10.6537408,5.3575520,8.879583,13.916346,NA,NA,NA,NA,NA


In [28]:
data_percentage = function(wider_dataset){
    # """
    # Calculating the studies or metals/chemicals with less than or equal to 20% of experimental data.

    # :param: wide dataframe
    # :output: a list that contains a dataframe of metals/chemicals and a dataframe of studies

    # """
    
    # converting values to 0s and 1s
    summed_df = wider_dataset %>%
        mutate(across(7:ncol(wider_dataset), ~ifelse(is.na(.x), 0, 1)))

    # identifying metals/chemicals w/ =< 20% of experimental data
    var_percentage_df = data.frame(Percent = colSums(summed_df[,7:ncol(summed_df)])/nrow(summed_df) * 100) %>%
        filter(Percent <= 40)
    # identifying studies w/ =< 20% of experimental data
    study_percentage_df = data.frame(HAWC_ID = wider_dataset$HAWC_ID,
                                     Percent = rowSums(summed_df[,7:ncol(summed_df)])/ncol(summed_df) * 100) %>%
        filter(Percent <= 40)

    return(list(var_percentage_df, study_percentage_df))
}

# calling fn
percentage_vol_df = data_percentage(wider_vol_df)
percentage_weight_df = data_percentage(wider_weight_df)

head(percentage_weight_df[[1]])

,Percent
,<dbl>
Ca,39.873418
Fe,39.240506
DTXSID9059757,25.316456
K,32.911392
Mg,29.113924
DTXSID1025649,5.696203


In [29]:
# removing those metals/chemicals & studies from the OG df
preimputed_vol_df = wider_vol_df[,colnames(wider_vol_df) %notin% rownames(percentage_vol_df[[1]])] 
preimputed_weight_df = wider_weight_df[,colnames(wider_weight_df) %notin% rownames(percentage_weight_df[[1]])]

preimputed_vol_df = preimputed_vol_df %>% filter(HAWC_ID %notin% percentage_vol_df[[2]]$HAWC_ID) 
preimputed_weight_df = preimputed_weight_df %>% filter(HAWC_ID %notin% percentage_weight_df[[2]]$HAWC_ID)

head(preimputed_vol_df)

SampleID,HAWC_ID,Metric,Sample,Study,Sample_Number,Cr,Cu,Pb,Ti,Zn,K,Ca,Fe,Mn,Ni,Sr
<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
WF_1289909_1,1289909,Volume,WF,Trieu et al. 2024,1,NA,2.744161,2.584963,5.087463,4.523562,NA,9.390169,8.233620,2.560715,1.000000,1.807355
WS_885041_1,885041,Volume,WS,Ihantola et al. 2020,1,5.865424,9.820179,6.930737,4.940167,13.538795,16.91999,11.761136,10.561288,7.266787,4.822730,4.078951
WS_885041_2,885041,Volume,WS,Ihantola et al. 2020,2,5.523562,8.640245,6.061776,5.281698,12.746304,16.19654,11.948002,9.383704,6.832890,4.655352,3.264536
WS_993956_1,993956,Volume,WS,Ihantola et al. 2022,1,6.285402,7.577429,6.584963,4.954196,14.053332,15.87270,11.451726,10.452241,8.134426,5.584963,4.392317
WS_1263480_1,1263480,Volume,WS,Burnet et al. 1990,1,NA,13.550867,12.425478,NA,18.116806,21.61807,18.892083,15.383671,13.550867,NA,12.773345
WS_1263480_2,1263480,Volume,WS,Burnet et al. 1990,2,NA,13.340963,NA,11.358651,20.306748,20.07464,18.694559,14.587895,13.074476,10.426265,12.720886


In [30]:
# How many metals/chemicals are now left in the dataset?
ncol(preimputed_vol_df) - 6
ncol(preimputed_weight_df) - 6

[1] 11

[1] 21

In [31]:
# How many study ids are left?
nrow(preimputed_vol_df)  
nrow(preimputed_weight_df) 

[1] 9

[1] 56

In [32]:
# transforming back to a long format
final_vol_df = preimputed_vol_df %>%
    pivot_longer(cols = 7:ncol(preimputed_vol_df), names_to = "DTXSID", values_to = "Value")
final_weight_df = preimputed_weight_df %>%
    pivot_longer(cols = 7:ncol(preimputed_weight_df), names_to = "DTXSID", values_to = "Value")

head(final_vol_df)

SampleID,HAWC_ID,Metric,Sample,Study,Sample_Number,DTXSID,Value
<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>
WF_1289909_1,1289909,Volume,WF,Trieu et al. 2024,1,Cr,NA
WF_1289909_1,1289909,Volume,WF,Trieu et al. 2024,1,Cu,2.744161
WF_1289909_1,1289909,Volume,WF,Trieu et al. 2024,1,Pb,2.584963
WF_1289909_1,1289909,Volume,WF,Trieu et al. 2024,1,Ti,5.087463
WF_1289909_1,1289909,Volume,WF,Trieu et al. 2024,1,Zn,4.523562
WF_1289909_1,1289909,Volume,WF,Trieu et al. 2024,1,K,NA


In [33]:
# adding back in name col
final_df = left_join(rbind(final_vol_df, final_weight_df), 
                      combined_df[,c(5,7:8)] %>%
    unique()) 

final_df = final_df[,c(4,2,5,6,9,3,7,10,8)]

head(final_df)
dim(final_df)

Joining with `by = join_by(DTXSID)`


Sample,HAWC_ID,Study,Sample_Number,Class,Metric,DTXSID,Name,Value
<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>
WF,1289909,Trieu et al. 2024,1,Metal,Volume,Cr,Chromium,NA
WF,1289909,Trieu et al. 2024,1,Metal,Volume,Cu,Copper,2.744161
WF,1289909,Trieu et al. 2024,1,Metal,Volume,Pb,Lead,2.584963
WF,1289909,Trieu et al. 2024,1,Metal,Volume,Ti,Titanium,5.087463
WF,1289909,Trieu et al. 2024,1,Metal,Volume,Zn,Zinc,4.523562
WF,1289909,Trieu et al. 2024,1,Metal,Volume,K,Potassium,NA


[1] 1275    9

In [34]:
# exporting
write.xlsx(final_df, paste0(Output,"/", "Processed_Data_", cur_date, ".xlsx"), 
           rowNames = FALSE)